In [1]:
# Import Pandas Library
import pandas as pd

# Create a path to the CSV files
csv_path = "Resources/schools_complete.csv"
csv_path2 = "Resources/students_complete.csv"

# Read CSV files into a Data Frame
schools = pd.read_csv(csv_path)
students = pd.read_csv(csv_path2)

# Combine data into single dataset
school_student = pd.merge(students, schools, how="left", on=["school_name", "school_name"])
school_student.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
# Data Frame operations to calculate the required District Summary
# Calculate the total number of schools
totalschools = len(school_student["school_name"].unique())

# Calculate the total number of students
totalstudents = len(school_student)

# Find the budget for each school and then add them together
budgets = school_student["budget"].unique()
totalbudget = budgets.sum()

# Calculate the average math score
averagemath = round(school_student["math_score"].mean(), 2)

# Calulate the average reading score
averagereading = round(school_student["reading_score"].mean(), 2)

# Calculate the percent of students passing math
passingmath = school_student.loc[school_student["math_score"] >= 70]
percentmath = round((len(passingmath)/totalstudents) * 100, 2)

# Calculate the percent of students passing reading
passingreading = school_student.loc[school_student["reading_score"] >= 70]
percentreading = round((len(passingreading)/totalstudents) * 100, 2)

# Calculate the percent of students passing both math and reading
passingboth = school_student.loc[(school_student["math_score"] >= 70) & (school_student["reading_score"] >= 70)]
percentboth = round((len(passingboth)/totalstudents) * 100, 2)

#Creates a Data Frame to display the above results
districtsummary = pd.DataFrame({
    "Total Schools": [totalschools],
    "Total Students": [totalstudents],
    "Total Budget": [totalbudget],
    "Average Math Score": [averagemath],
    "Average Reading Score": [averagereading],
    "% Passing Math": [percentmath],
    "% Passing Reading": [percentreading],
    "% Overall Passing": [percentboth]
})
districtsummary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,81.88,74.98,85.81,65.17


In [3]:
# Sorts data frame into alphabetical order by school name and renames column to appropriate format
schoolssorted = schools.sort_values("school_name")
schoolssorted = schoolssorted.rename(columns={"school_name":"School Name"})
schoolssorted.head()

,School ID,School Name,type,size,budget
7,7,Bailey High School,District,4976,3124928
6,6,Cabrera High School,Charter,1858,1081356
1,1,Figueroa High School,District,2949,1884411
13,13,Ford High School,District,2739,1763916
4,4,Griffin High School,Charter,1468,917500


In [4]:
# Pulls out the total number of students at each school into a series
totalstudents = schoolssorted.set_index("School Name")["size"]

# Pulls out the type of each school into a series
schooltype = schoolssorted.set_index("School Name")["type"]

# Pulls out the budget of each school into a series
totalbudgets = schoolssorted.set_index("School Name")["budget"]

# Calculates the budget per student at the school
perstudentbudget = totalbudgets/totalstudents

# Creates a dataframe of all the series calculated above
schoolsummary = pd.DataFrame({
    "School Type": schooltype, "Total Students": totalstudents, "Total School Budget": totalbudgets,
    "Per Student Budget": perstudentbudget
})
schoolsummary

,School Type,Total Students,Total School Budget,Per Student Budget
School Name,,,,
Bailey High School,District,4976,3124928,628.0
Cabrera High School,Charter,1858,1081356,582.0
Figueroa High School,District,2949,1884411,639.0
Ford High School,District,2739,1763916,644.0
Griffin High School,Charter,1468,917500,625.0
Hernandez High School,District,4635,3022020,652.0
Holden High School,Charter,427,248087,581.0
Huang High School,District,2917,1910635,655.0
Johnson High School,District,4761,3094650,650.0


In [5]:
# Groups the data frame by school
schoolsv2 = school_student.groupby(["school_name"])

# Pulls out the total number of students at each school into a data frame
totalstudents = schoolsv2[["student_name"]].count()

# Pulls out the budget, average math and reading scores at each school into a data frame
averages = schoolsv2[["budget", "math_score", "reading_score"]].mean()

# Merges the 2 data frames above into one
df = pd.merge(totalstudents, averages, on="school_name")

# Adds budget per student column to the data frame
df["Per Student Budget"] = df["budget"]/df["student_name"]

# Adds the % of students passing math from each school to data frame
passingmathv2 = passingmath.groupby(["school_name"])
mathstudents = passingmathv2[["student_name"]].count()
df["% Passing Math"] = (mathstudents["student_name"]/df["student_name"])*100

# Adds the % of students passing reading from each school to data frame
passingreadingv2 = passingreading.groupby(["school_name"])
readingstudents = passingreadingv2[["student_name"]].count()
df["% Passing Reading"] = (readingstudents["student_name"]/df["student_name"])*100

# Adds the % of students passing overall from each school to data frame
passingbothv2 = passingboth.groupby(["school_name"])
bothstudents = passingbothv2[["student_name"]].count()
df["% Overall Passing"] = (bothstudents["student_name"]/df["student_name"])*100

# Adds the type of school to the data frame
df["School Type"] = schoolsv2["type"].unique()
# df.astype({"School Type": "string"}).dtypes ------- trying to convert to string for later use
# test = schools.sort_values("school_name")
# df["School Type"] = test["type"]
# df.append(test["type"])

# Creates a new data frame with the correct column titles
schoolsummary = df.rename(columns={
    "student_name":"Total Students", "budget":"Total School Budget", "math_score":"Average Math Score",
    "reading_score":"Average Reading Score", "type": "School Type"
})

# Reorders the data frame to the correct column order
schoolsummary = schoolsummary[["School Type","Total Students","Total School Budget","Per Student Budget",
                              "Average Math Score","Average Reading Score","% Passing Math",
                              "% Passing Reading","% Overall Passing"]]
schoolsummary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,[District],4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,[Charter],1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,[District],2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,[District],2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,[Charter],1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,[District],4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,[Charter],427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,[District],2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,[District],4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [6]:
# Displays the top 5 schools sorted by % Overall Passing
topperformers = schoolsummary.sort_values("% Overall Passing", ascending=False)
topperformers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,[Charter],1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,[Charter],1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,[Charter],1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,[Charter],2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,[Charter],962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [7]:
# Displays the bottom 5 schools sorted by % Overall Passing
bottomperformers = schoolsummary.sort_values("% Overall Passing")
bottomperformers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,[District],3999,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,[District],2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,[District],2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,[District],4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,[District],4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [8]:
# Groups original Data Frame by school name and then grade
schoolsbygrade = school_student.groupby(["school_name", "grade"])

ninthgrademath = []
tenthgrademath= []
eleventhgrademath= []
twelfthgrademath= []

# Pulls out a series showing the average math score sorted by school and grade
mathaverage_bygrade = schoolsbygrade.mean()
mathaverage_bygrade = mathaverage_bygrade["math_score"]
mathaverage_bygrade

school_name            grade
Bailey High School     10th     76.996772
                       11th     77.515588
                       12th     76.492218
                       9th      77.083676
Cabrera High School    10th     83.154506
                       11th     82.765560
                       12th     83.277487
                       9th      83.094697
Figueroa High School   10th     76.539974
                       11th     76.884344
                       12th     77.151369
                       9th      76.403037
Ford High School       10th     77.672316
                       11th     76.918058
                       12th     76.179963
                       9th      77.361345
Griffin High School    10th     84.229064
                       11th     83.842105
                       12th     83.356164
                       9th      82.044010
Hernandez High School  10th     77.337408
                       11th     77.136029
                       12th     77.186567
     

In [9]:
# Pulls out a series showing the average reading score sorted by school and grade
readingaverage_bygrade = schoolsbygrade.mean("reading_score")
readingaverage_bygrade = readingaverage_bygrade["reading_score"]
readingaverage_bygrade

school_name            grade
Bailey High School     10th     80.907183
                       11th     80.945643
                       12th     80.912451
                       9th      81.303155
Cabrera High School    10th     84.253219
                       11th     83.788382
                       12th     84.287958
                       9th      83.676136
Figueroa High School   10th     81.408912
                       11th     80.640339
                       12th     81.384863
                       9th      81.198598
Ford High School       10th     81.262712
                       11th     80.403642
                       12th     80.662338
                       9th      80.632653
Griffin High School    10th     83.706897
                       11th     84.288089
                       12th     84.013699
                       9th      83.369193
Hernandez High School  10th     80.660147
                       11th     81.396140
                       12th     80.857143
     

In [10]:
# Create bins for the amount of money spent per student
b = [0, 585, 630, 645, 680]
groupnames = ["<$585", "$585-630", "$630-645", "$645-680"]

# Adds per student spending column to previous school summary data frame
schoolsummary["Spending Ranges (Per Student)"] = pd.cut(schoolsummary["Per Student Budget"], bins=b, labels=groupnames,
                                                       include_lowest=True)

# Groups data frame by the spending ranges outlined in the bins 
spendingranges = schoolsummary.groupby(["Spending Ranges (Per Student)"])

# Grabs the mean value of necessary columns in data frame and outputs the resulting data frame
scoresbyspending = spendingranges.mean()
scoresbyspending = scoresbyspending[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
scoresbyspending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


In [11]:
# Create bins for the size of the school by student count
b2 = [0, 1000, 2000, 5000]
groupnames2 = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

# Adds school size column to previous school summary data frame
schoolsummary["School Size"] = pd.cut(schoolsummary["Total Students"], bins=b2, labels=groupnames2,
                                                       include_lowest=True)

# Groups data frame by the school size outlined in the bins 
schoolsize = schoolsummary.groupby(["School Size"])

# Grabs the mean value of necessary columns in data frame and outputs the resulting data frame
scoresbysize = schoolsize.mean()
scoresbysize = scoresbysize[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
scoresbysize

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [12]:
# Groups school summary data frame by school type
schooltype = schoolsummary.groupby(["School Type"])

# Grabs the mean value of necessary columns in data frame and outputs the resulting data frame
# scoresbytype = schooltype.mean()
# scoresbytype = scoresbytype[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
# scoresbytype